In [1]:
import numpy as np
import matplotlib.pyplot as plt
import validator as val

from data_handler import DataHandler
from log_regression import LogisticalRegression
from calculations import Calculations

dh = DataHandler("spambase.data")
lr = LogisticalRegression()
calc = Calculations()

In [2]:
# Pull, Split, Score, and SplitXY Data
# X is an N row by D column matrix, where N is hte number of observations and D is the number of features.  
# Y is an N by 1 column vector.  
# W is a D by 1 column vector.
data = dh.parse_data_no_header()
data = dh.shuffle_data(data)
data_train, data_test = dh.split_data(data)
tX, tY = dh.getXY(data_train, -1, -1)
vX, vY = dh.getXY(data_test, -1, -1)
# mean, std = dh.zscores(tX)
# accuracy = (true_values == predictions).sum() / N

mean, std = dh.zscore_data(tX, tX)
test = val.validate_zscore(tX)
tX = dh.apply_zscore(mean, std, tX)
vX = dh.apply_zscore(mean, std, vX)
print("Accuracy of ZScore={} %".format(100-(test == tX).sum()/tX.shape[0]))

# vX = dh.zscore_data(tX, vX)
tX.shape, tY.shape, vX.shape, vY.shape

Accuracy of ZScore=99.99250081512879 %


((3067, 57), (3067, 1), (1533, 57), (1533, 1))

In [3]:
# X is an M row by N column matrix, where M is hte number of observations and N is the number of features.  
# Y is an M by 1 column vector.  
# W is a N by 1 column vector.
m, n = tX.shape
w = np.zeros((n, 1))
b = 0

In [ ]:
# calculate(self, w, b, tX, tY, vX, vY):
weights, bias, losses = lr.calculate(w, b, tX, tY, vX, vY)

TMean Cost after iteration 0: 0.346574
VMean Cost after iteration 0: 0.693147
TMean Cost after iteration 100: 0.705460
VMean Cost after iteration 100: 0.726053


In [ ]:
# def prediction(self, w, b, X):
print(tX)
tPreds = lr.prediction(weights, bias, tX)
vPreds = lr.prediction(weights, bias, vX)

print("Training Accuracy: {} %".format(calculations.accuracy(tY, tPreds)))
print("Training Precision: {} %".format(calculations.precision(tY, tPreds)))
print("Validation Accuracy: {} %".format(calculations.accuracy(vY, vPreds)))
print("Validation Precision: {} %".format(calculations.precision(vY, vPreds)))